In [1]:
import numpy as np
from PIL import ImageGrab
import cv2
import matplotlib.pyplot as plt
import os
from time import sleep
from sklearn.cluster import DBSCAN

In [5]:
image_files = ["mapped_data\\" + fname for fname in os.listdir('mapped_data') if fname[-5:] != 'd.jpg']
map_files = ["mapped_data\\" + fname for fname in os.listdir('mapped_data') if fname[-5:] == 'd.jpg']

In [7]:
imgs = [plt.imread(fname) for fname in image_files]
map_imgs = [plt.imread(fname) for fname in map_files]

In [9]:



def roi(img, vertices):
    #blank mask:
    mask = np.zeros_like(img)
    # fill the mask
    cv2.fillPoly(mask, vertices, 255)
    # now only show the area that is the mask
    masked = cv2.bitwise_and(img, mask)
    return masked

def erase_region(img,vertices):
    #blank mask:
    mask = np.zeros_like(img)
    mask = mask + 255
    # fill the mask
    cv2.fillPoly(mask, vertices, 0)
    # now only show the area that is the mask
    masked = cv2.bitwise_and(img, mask)
    return masked
    

def draw_lines(img,lines,color = [255,255,255]):
    if lines is not None:
        for coords in lines:
            coords = coords.astype(np.int32)
            cv2.line(img, (coords[0], coords[1]), (coords[2], coords[3]), color, 3)

def process_img(image,debug=False,showHough = False,bfilter1=5,bfilter2=75,bfilter3=75,canny_tr1 = 100, canny_tr2 = 100, 
               rho = 0.7,theta = 1*np.pi/180,threshold = 100,minLineLength = 200,maxLineGap = 100):

    if debug:
        plt.figure(figsize=(20,20))
        plt.imshow(image)
        plt.show()
        sleep(2)
        
    image = cv2.bilateralFilter(image,bfilter1,bfilter2,bfilter3)
    
    if debug:
        plt.figure(figsize=(20,20))
        plt.imshow(image)
        plt.show()
        sleep(0.5)
 
    processed_img =  cv2.Canny(image, threshold1 = canny_tr1, threshold2=canny_tr2)
    if debug:
        plt.figure(figsize=(20,20))
        plt.imshow(processed_img)
        plt.show()
        sleep(0.5)
    
    h,w = image.shape[:2]
    
    vertices = np.array([[0,780],[0,600],[480,350],[1440,350],[w,600],[w,780]], np.int32)
    
    car = np.array([[500,h],[850,450],[1050,450],[1500,h]],np.int32)
    
#     gui = np.array([[0,900],[300,900],[300,1080],[0,1080]],np.int32)
    
    processed_img = roi(processed_img, [vertices])
    processed_img = erase_region(processed_img, [car])
#     processed_img = erase_region(processed_img,[gui] )

    if debug:
        plt.figure(figsize=(20,20))
        plt.imshow(processed_img)
        plt.show()
        sleep(0.5)
    
    lines = cv2.HoughLinesP(processed_img,rho = rho,theta = theta,threshold = threshold,minLineLength = minLineLength,maxLineGap = maxLineGap)
    if lines is not None:
        lines = lines[:,0,:]
        line_angles = np.arctan2(lines[:,2] - lines[:,0] , lines[:,1] - lines[:,3])*180/3.14
        lines = lines[(line_angles  > 100) | (line_angles  < 80) ]
     
    if debug or showHough:
        plt.figure(figsize=(20,20))
        draw_lines(processed_img,lines)
        plt.imshow(processed_img)
        plt.show()
        sleep(0.5)
    
    return lines

def getBorderLines(img,debug=False,eps=0.1,*args,**kwargs):
    lines = process_img(img,debug,*args,**kwargs)
    clst = DBSCAN(eps=eps)
    angles = np.arctan2(lines[:,2] - lines[:,0] , lines[:,1] - lines[:,3])
    preds = clst.fit_predict(angles[:,None])
    print(preds)
    line1,line2 = None,None
    if (preds==0).any():
        line1 = lines[preds==0].mean(axis=0)
    if (preds==1).any():
        line2 = lines[preds==1].mean(axis=0)
   
    return line1,line2

def extend_lines(lines,scale):
    extended = []
    for line in lines:
        if line is not None:
            p1 = line[:2]
            p2 = line[2:]
            x1,y1 = (p1 + p2)*0.5 - (p2 - p1) * scale*0.5
            x2,y2 = (p1 + p2)*0.5 + (p2 - p1) * scale*0.5
            extended.append(np.array([x1,y1,x2,y2]))
    return extended

def limit_lines(lines, limit):
    results = []
    for line in lines:
        if line is not None:
            y_max_index = np.argmax(line[1::2])
            if y_max_index == 0:
                p1 = line[[0,1]]
                p2 = line[[2,3]]
            else:
                p2 = line[[0,1]]
                p1 = line[[2,3]]

            l = (p2 - p1)
            k = l[1]/l[0]
            x= p1[0] + -(p1[1] - limit)/k

            results.append(np.array([*p1,x,limit]))
    return results
        